In [1]:
import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OrdinalEncoder

import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras.layers import SimpleRNN, LSTM, Dense
from keras.layers import Dropout
from keras.layers import Reshape
from keras.models import Model
from keras.optimizers import Adam
from keras.layers import BatchNormalization

import warnings
warnings.filterwarnings('ignore')

In [2]:
oil = pd.read_csv((r'C:\Users\sahithi aharam\Downloads\oil.csv'),index_col='date')
holidays_events= pd.read_csv((r'C:\Users\sahithi aharam\Downloads\holidays_events.csv'),index_col='date')
stores= pd.read_csv(r'C:\Users\sahithi aharam\Downloads\stores.csv')

transactions = pd.read_csv(r'C:\Users\sahithi aharam\Downloads\transactions.csv\transactions.csv')
#df_submission = pd.read_csv(r'C:\Users\sahithi aharam\Downloads\sample_submission.csv')

train = pd.read_csv((r'C:\Users\sahithi aharam\Downloads\train(1).csv\train.csv'),index_col='id',parse_dates=['date'], infer_datetime_format=True)
test = pd.read_csv((r'C:\Users\sahithi aharam\Downloads\test (1).csv'),parse_dates=['date'], infer_datetime_format=True)

# Data preprocessing

In [3]:
train["family"].nunique(dropna = True) 

33

In [4]:
#dropping unused columns
train_data = train.copy().drop(['onpromotion'], axis=1)
test_data = test.copy().drop(['onpromotion'], axis=1)

In [5]:
##applying the encoders, the data showed that there are ~33 unique elements
ordinal_encoder = OrdinalEncoder(dtype=int)
train_data[['family']] = ordinal_encoder.fit_transform(train_data[['family']])
test_data[['family']] = ordinal_encoder.transform(test_data[['family']])

In [6]:
##counting the number of days
n_o_days_train=train["date"].nunique(dropna = False) 
print('number of day train:',n_o_days_train)

# number of store
n_o_stores_train=train["store_nbr"].nunique(dropna = False) 
print('number of stores train:',n_o_stores_train)

# number of family
n_o_families_train=train["family"].nunique(dropna = False) 
print('number of family/type of prod train:',n_o_families_train)

number of day train: 1684
number of stores train: 54
number of family/type of prod train: 33


In [7]:
##counting the number of days
n_o_days_test=test["date"].nunique(dropna = False) 
print('number of day test:',n_o_days_test)

# number of store
n_o_stores_test=test["store_nbr"].nunique(dropna = False) 
print('number of stores test:',n_o_stores_test)

# number of family
n_o_families_test=test["family"].nunique(dropna = False) 
print('number of family/type of prod test:',n_o_families_test)

number of day test: 16
number of stores test: 54
number of family/type of prod test: 33


In [8]:
pivoted_train = train_data.pivot(index=['date'], columns=['store_nbr', 'family'], values='sales')
pivoted_train

store_nbr     1                                                         \
family       0    1    2       3    4          5     6       7      8    
date                                                                     
2013-01-01  0.0  0.0  0.0     0.0  0.0    0.00000   0.0     0.0    0.0   
2013-01-02  2.0  0.0  2.0  1091.0  0.0  470.65200   0.0  1060.0  579.0   
2013-01-03  3.0  0.0  0.0   919.0  0.0  310.65500   0.0   836.0  453.0   
2013-01-04  3.0  0.0  3.0   953.0  0.0  198.36600   0.0   827.0  460.0   
2013-01-05  5.0  0.0  3.0  1160.0  0.0  301.05700   0.0   811.0  464.0   
...         ...  ...  ...     ...  ...        ...   ...     ...    ...   
2017-08-11  1.0  0.0  1.0  1006.0  0.0  145.60700   4.0   341.0  343.0   
2017-08-12  6.0  0.0  3.0  1659.0  0.0  243.22000   3.0   351.0  526.0   
2017-08-13  1.0  0.0  1.0   803.0  0.0  136.67900   1.0   169.0  266.0   
2017-08-14  1.0  0.0  6.0  2201.0  0.0  346.03800   4.0   571.0  699.0   
2017-08-15  4.0  0.0  4.0  1942.0  0.0  329.54102  21.0   703.0  602.0   

store_nbr            ...     9                                       \
family           9   ...    23       24     25    26    27       28   
date                 ...                                              
2013-01-01    0.000  ...   0.0    0.000    0.0   0.0   0.0    0.000   
2013-01-02  164.069  ...   0.0  374.531  482.0   0.0   0.0  651.292   
2013-01-03  151.582  ...   0.0  400.863  372.0   0.0   0.0  509.496   
2013-01-04  131.411  ...   0.0  310.878  324.0   0.0   0.0  332.672   
2013-01-05  118.613  ...   0.0  447.685  461.0   0.0   0.0  510.919   
...             ...  ...   ...      ...    ...   ...   ...      ...   
2017-08-11   64.302  ...   5.0  309.245  373.0  11.0   2.0  525.224   
2017-08-12   99.488  ...   2.0  260.298  400.0   7.0  10.0  383.387   
2017-08-13   47.770  ...   3.0  327.206  510.0   2.0   9.0  412.458   
2017-08-14  154.578  ...  12.0  330.975  445.0   2.0  14.0  283.429   
2017-08-15  116.402  ...  11.0  449.228  522.0   6.0   6.0  438.133   

store_nbr                                          
family             29        30     31         32  
date                                               
2013-01-01    0.00000     0.000    0.0   0.000000  
2013-01-02   83.00000     0.000    0.0  29.214000  
2013-01-03   66.00000     0.000    0.0  25.000000  
2013-01-04   57.00000     0.000    0.0  11.000000  
2013-01-05   84.00000     0.000    0.0  28.361000  
...               ...       ...    ...        ...  
2017-08-11  112.10000  1453.078  140.0  23.831000  
2017-08-12  129.90399  1419.264  138.0  16.859001  
2017-08-13  105.16900  1693.607  200.0  20.000000  
2017-08-14  114.12000  1348.425  182.0  17.000000  
2017-08-15  154.55300  2419.729  121.0  16.000000  

[1684 rows x 1782 columns]

In [9]:
train_samples = int(n_o_days_train * 0.95)

In [10]:
train_samples_df = pivoted_train[:train_samples]
train_samples_df

store_nbr     1                                                                \
family       0    1    2       3    4        5     6       7      8        9    
date                                                                            
2013-01-01  0.0  0.0  0.0     0.0  0.0    0.000   0.0     0.0    0.0    0.000   
2013-01-02  2.0  0.0  2.0  1091.0  0.0  470.652   0.0  1060.0  579.0  164.069   
2013-01-03  3.0  0.0  0.0   919.0  0.0  310.655   0.0   836.0  453.0  151.582   
2013-01-04  3.0  0.0  3.0   953.0  0.0  198.366   0.0   827.0  460.0  131.411   
2013-01-05  5.0  0.0  3.0  1160.0  0.0  301.057   0.0   811.0  464.0  118.613   
...         ...  ...  ...     ...  ...      ...   ...     ...    ...      ...   
2017-05-18  4.0  0.0  2.0  2154.0  0.0  350.502  12.0   669.0  718.0  132.686   
2017-05-19  5.0  0.0  4.0  2743.0  0.0  481.324  24.0   826.0  957.0  177.191   
2017-05-20  4.0  0.0  3.0  2379.0  0.0  392.965   7.0   588.0  795.0  140.261   
2017-05-21  1.0  0.0  1.0   974.0  0.0  161.529   1.0   243.0  337.0   75.683   
2017-05-22  2.0  0.0  5.0  2620.0  1.0  418.029  16.0   712.0  754.0  129.027   

store_nbr   ...    9                                                       \
family      ...   23         24     25    26    27         28          29   
date        ...                                                             
2013-01-01  ...  0.0    0.00000    0.0   0.0   0.0    0.00000    0.000000   
2013-01-02  ...  0.0  374.53100  482.0   0.0   0.0  651.29200   83.000000   
2013-01-03  ...  0.0  400.86300  372.0   0.0   0.0  509.49600   66.000000   
2013-01-04  ...  0.0  310.87800  324.0   0.0   0.0  332.67200   57.000000   
2013-01-05  ...  0.0  447.68500  461.0   0.0   0.0  510.91900   84.000000   
...         ...  ...        ...    ...   ...   ...        ...         ...   
2017-05-18  ...  4.0  541.15700  344.0  19.0  12.0  313.56600   73.060000   
2017-05-19  ...  2.0  264.74400  268.0   6.0   8.0  477.29602   79.421000   
2017-05-20  ...  5.0  373.00400  569.0  10.0  12.0  503.79500  151.274000   
2017-05-21  ...  2.0  492.31302  636.0  10.0  18.0  688.77600  149.067000   
2017-05-22  ...  4.0  319.78200  345.0   8.0   9.0  321.63200  111.423004   

store_nbr                              
family             30   31         32  
date                                   
2013-01-01     0.0000  0.0   0.000000  
2013-01-02     0.0000  0.0  29.214000  
2013-01-03     0.0000  0.0  25.000000  
2013-01-04     0.0000  0.0  11.000000  
2013-01-05     0.0000  0.0  28.361000  
...               ...  ...        ...  
2017-05-18  1262.8600  7.0  12.544001  
2017-05-19  1149.9869  2.0   9.238000  
2017-05-20  2008.8520  3.0  27.380001  
2017-05-21  2424.7440  7.0  39.710000  
2017-05-22  1413.0350  2.0  19.097000  

[1599 rows x 1782 columns]

In [11]:
valid_samples_df = pivoted_train[train_samples:]
valid_samples_df

store_nbr     1                                                        \
family       0    1    2       3    4          5     6      7      8    
date                                                                    
2017-05-23  4.0  0.0  1.0  2365.0  1.0  413.15200   8.0  822.0  736.0   
2017-05-24  7.0  0.0  1.0  2611.0  1.0  349.68100  11.0  944.0  867.0   
2017-05-25  2.0  0.0  1.0  2229.0  0.0  372.38500  18.0  706.0  723.0   
2017-05-26  1.0  0.0  0.0  1100.0  0.0  190.73300   7.0  303.0  375.0   
2017-05-27  1.0  0.0  2.0  2270.0  0.0  300.78600  10.0  462.0  634.0   
...         ...  ...  ...     ...  ...        ...   ...    ...    ...   
2017-08-11  1.0  0.0  1.0  1006.0  0.0  145.60700   4.0  341.0  343.0   
2017-08-12  6.0  0.0  3.0  1659.0  0.0  243.22000   3.0  351.0  526.0   
2017-08-13  1.0  0.0  1.0   803.0  0.0  136.67900   1.0  169.0  266.0   
2017-08-14  1.0  0.0  6.0  2201.0  0.0  346.03800   4.0  571.0  699.0   
2017-08-15  4.0  0.0  4.0  1942.0  0.0  329.54102  21.0  703.0  602.0   

store_nbr            ...     9                                         \
family           9   ...    23       24     25    26    27         28   
date                 ...                                                
2017-05-23  127.909  ...   1.0  284.679  340.0   5.0  12.0  328.32900   
2017-05-24  166.068  ...   1.0  318.254  292.0   6.0  12.0  360.76498   
2017-05-25  122.562  ...   1.0  415.299  260.0   7.0  10.0  261.97600   
2017-05-26   82.697  ...   5.0  378.795  382.0   9.0   9.0  533.20300   
2017-05-27   99.587  ...   3.0  412.021  455.0   8.0  11.0  541.38000   
...             ...  ...   ...      ...    ...   ...   ...        ...   
2017-08-11   64.302  ...   5.0  309.245  373.0  11.0   2.0  525.22400   
2017-08-12   99.488  ...   2.0  260.298  400.0   7.0  10.0  383.38700   
2017-08-13   47.770  ...   3.0  327.206  510.0   2.0   9.0  412.45800   
2017-08-14  154.578  ...  12.0  330.975  445.0   2.0  14.0  283.42900   
2017-08-15  116.402  ...  11.0  449.228  522.0   6.0   6.0  438.13300   

store_nbr                                           
family             29         30     31         32  
date                                                
2017-05-23  127.40600  2268.3320    1.0  16.063000  
2017-05-24  104.61800  1180.0780    2.0  18.163000  
2017-05-25  110.95700   920.1430    5.0   2.994000  
2017-05-26  169.91900  1515.1750   13.0  20.683000  
2017-05-27  134.74800  1634.6849    5.0  43.512000  
...               ...        ...    ...        ...  
2017-08-11  112.10000  1453.0780  140.0  23.831000  
2017-08-12  129.90399  1419.2640  138.0  16.859001  
2017-08-13  105.16900  1693.6070  200.0  20.000000  
2017-08-14  114.12000  1348.4250  182.0  17.000000  
2017-08-15  154.55300  2419.7290  121.0  16.000000  

[85 rows x 1782 columns]

In [12]:
minmax_scaler = MinMaxScaler()
minmax_scaler.fit(train_samples_df)

scaled_train_samples = minmax_scaler.transform(train_samples_df)
scaled_validation_samples = minmax_scaler.transform(valid_samples_df)

In [13]:
def split_series(series, n_past, n_future):
  X, y = list(), list()
  for window_start in range(len(series)):
    past_end = window_start + n_past
    future_end = past_end + n_future
    if future_end > len(series):
      break
    # slicing the past and future parts of the window
    past, future = series[window_start:past_end, :], series[past_end:future_end, :]
    X.append(past)
    y.append(future)
  return np.array(X), np.array(y)

n_past =4
n_future = 4
n_features = n_o_stores_train * n_o_families_train # num of features

In [14]:
X_train, y_train = split_series(scaled_train_samples,n_past, n_future)
X_val, y_val = split_series(scaled_validation_samples,n_past, n_future)

# Model

In [15]:
from keras.regularizers import l1, l2
def timemodel():
    model = keras.Sequential()
### basic RNN model
    model.add(layers.SimpleRNN(units=200, activation='relu', return_sequences=True, input_shape=(n_past,n_features),kernel_regularizer=l1(0.01)))
    model.add(keras.layers.BatchNormalization())
    model.add(Dropout(0.3))
    model.add(layers.SimpleRNN(units=200,activation='relu', return_sequences=True,kernel_regularizer=l2(0.001)))
    model.add(keras.layers.BatchNormalization())
    model.add(Dropout(0.2))
    model.add(keras.layers.TimeDistributed(keras.layers.Dense(n_features)))
    
    
    model.compile(loss="mae", optimizer=keras.optimizers.Adam(learning_rate=0.001), metrics=['mae'])
    return model

In [16]:
model=timemodel()
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn (SimpleRNN)      (None, 4, 200)            396600    
                                                                 
 batch_normalization (BatchN  (None, 4, 200)           800       
 ormalization)                                                   
                                                                 
 dropout (Dropout)           (None, 4, 200)            0         
                                                                 
 simple_rnn_1 (SimpleRNN)    (None, 4, 200)            80200     
                                                                 
 batch_normalization_1 (Batc  (None, 4, 200)           800       
 hNormalization)                                                 
                                                                 
 dropout_1 (Dropout)         (None, 4, 200)            0

In [17]:

early_stopping = keras.callbacks.EarlyStopping(monitor = 'val_mae',
                                               min_delta=0.001,
                                               patience=100, 
                                               restore_best_weights=True)

EPOCHS = 100
model_history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val), 
    epochs=EPOCHS, 
    callbacks=[early_stopping], 
    batch_size=256, 
    shuffle=True)

Epoch 1/100
7/7 [==============================] - 5s 236ms/step - loss: 89.8056 - mae: 0.3949 - val_loss: 75.5827 - val_mae: 0.2770
Epoch 2/100
7/7 [==============================] - 1s 108ms/step - loss: 67.9676 - mae: 0.2909 - val_loss: 56.0009 - val_mae: 0.3262
Epoch 3/100
7/7 [==============================] - 1s 94ms/step - loss: 49.4732 - mae: 0.2323 - val_loss: 39.5055 - val_mae: 0.3029
Epoch 4/100
7/7 [==============================] - 1s 114ms/step - loss: 34.1247 - mae: 0.2035 - val_loss: 26.0526 - val_mae: 0.2424
Epoch 5/100
7/7 [==============================] - 1s 111ms/step - loss: 21.8551 - mae: 0.1820 - val_loss: 15.6399 - val_mae: 0.1839
Epoch 6/100
7/7 [==============================] - 1s 97ms/step - loss: 12.6367 - mae: 0.1600 - val_loss: 8.3647 - val_mae: 0.1563
Epoch 7/100
7/7 [==============================] - 1s 104ms/step - loss: 6.5164 - mae: 0.1363 - val_loss: 4.1540 - val_mae: 0.1395
Epoch 8/100
7/7 [==============================] - 1s 113ms/step - loss: 3

In [18]:
x_test_pred = scaled_validation_samples[-n_past:,:].reshape((1, n_past, n_features))
print(x_test_pred.shape)
scaled_y_predict = model.predict(x_test_pred)

(1, 4, 1782)


1/1 [==============================] - 1s 1s/step


In [19]:
y_predict = pd.DataFrame(minmax_scaler.inverse_transform(scaled_y_predict.reshape((n_future, n_features))),
                         columns=valid_samples_df.columns)
## each element corresponding to raw value at a specific ID no on pivoted test table 
y_predict

store_nbr         1                                                         \
family           0         1         2            3         4           5    
0          3.917730 -0.000398  2.956953  2456.177734 -0.001714  420.304932   
1          3.591026 -0.001257  3.103729  2283.461182 -0.010339  414.156494   
2          3.764398 -0.001288  3.082764  2336.047852 -0.008086  421.731140   
3          3.781224 -0.001607  3.136055  2384.667725 -0.009875  425.155304   

store_nbr                                                 ...         9  \
family            6           7           8           9   ...        23   
0          23.531656  733.502441  806.646179  142.905304  ...  3.349048   
1          19.393139  696.046265  775.801880  134.592880  ...  3.603992   
2          20.384672  715.614136  801.045410  144.893265  ...  3.761200   
3          20.338955  710.895935  816.449768  144.195984  ...  3.982051   

store_nbr                                                           \
family             24          25        26         27          28   
0          380.883636  435.569244  6.111869  11.627018  399.358795   
1          423.137482  442.095459  6.129599  11.379129  381.907806   
2          379.110626  426.957855  6.203785  11.244270  393.960602   
3          383.049316  456.925873  6.530647  11.712188  408.949005   

store_nbr                                                
family             29           30        31         32  
0          102.809708  1553.903687  2.814135  13.829038  
1          109.531868  1519.845215  3.793168  14.174432  
2          107.283676  1501.896118  3.714189  13.883819  
3          112.429947  1603.354248  4.022950  14.500783  

[4 rows x 1782 columns]

In [20]:
pivoted_test = test_data.pivot(index=['date'], columns=['store_nbr', 'family'])
pivoted_test ## format store_nbr, family, date and each value/element corresponding to the sample indices

id                                                        \
store_nbr         1                                                         
family           0        1        2        3        4        5        6    
date                                                                        
2017-08-16  3000888  3000889  3000890  3000891  3000892  3000893  3000894   
2017-08-17  3002670  3002671  3002672  3002673  3002674  3002675  3002676   
2017-08-18  3004452  3004453  3004454  3004455  3004456  3004457  3004458   
2017-08-19  3006234  3006235  3006236  3006237  3006238  3006239  3006240   
2017-08-20  3008016  3008017  3008018  3008019  3008020  3008021  3008022   
2017-08-21  3009798  3009799  3009800  3009801  3009802  3009803  3009804   
2017-08-22  3011580  3011581  3011582  3011583  3011584  3011585  3011586   
2017-08-23  3013362  3013363  3013364  3013365  3013366  3013367  3013368   
2017-08-24  3015144  3015145  3015146  3015147  3015148  3015149  3015150   
2017-08-25  3016926  3016927  3016928  3016929  3016930  3016931  3016932   
2017-08-26  3018708  3018709  3018710  3018711  3018712  3018713  3018714   
2017-08-27  3020490  3020491  3020492  3020493  3020494  3020495  3020496   
2017-08-28  3022272  3022273  3022274  3022275  3022276  3022277  3022278   
2017-08-29  3024054  3024055  3024056  3024057  3024058  3024059  3024060   
2017-08-30  3025836  3025837  3025838  3025839  3025840  3025841  3025842   
2017-08-31  3027618  3027619  3027620  3027621  3027622  3027623  3027624   

                                       ...                             \
store_nbr                              ...        9                     
family           7        8        9   ...       23       24       25   
date                                   ...                              
2017-08-16  3000895  3000896  3000897  ...  3002660  3002661  3002662   
2017-08-17  3002677  3002678  3002679  ...  3004442  3004443  3004444   
2017-08-18  3004459  3004460  3004461  ...  3006224  3006225  3006226   
2017-08-19  3006241  3006242  3006243  ...  3008006  3008007  3008008   
2017-08-20  3008023  3008024  3008025  ...  3009788  3009789  3009790   
2017-08-21  3009805  3009806  3009807  ...  3011570  3011571  3011572   
2017-08-22  3011587  3011588  3011589  ...  3013352  3013353  3013354   
2017-08-23  3013369  3013370  3013371  ...  3015134  3015135  3015136   
2017-08-24  3015151  3015152  3015153  ...  3016916  3016917  3016918   
2017-08-25  3016933  3016934  3016935  ...  3018698  3018699  3018700   
2017-08-26  3018715  3018716  3018717  ...  3020480  3020481  3020482   
2017-08-27  3020497  3020498  3020499  ...  3022262  3022263  3022264   
2017-08-28  3022279  3022280  3022281  ...  3024044  3024045  3024046   
2017-08-29  3024061  3024062  3024063  ...  3025826  3025827  3025828   
2017-08-30  3025843  3025844  3025845  ...  3027608  3027609  3027610   
2017-08-31  3027625  3027626  3027627  ...  3029390  3029391  3029392   

                                                                           
store_nbr                                                                  
family           26       27       28       29       30       31       32  
date                                                                       
2017-08-16  3002663  3002664  3002665  3002666  3002667  3002668  3002669  
2017-08-17  3004445  3004446  3004447  3004448  3004449  3004450  3004451  
2017-08-18  3006227  3006228  3006229  3006230  3006231  3006232  3006233  
2017-08-19  3008009  3008010  3008011  3008012  3008013  3008014  3008015  
2017-08-20  3009791  3009792  3009793  3009794  3009795  3009796  3009797  
2017-08-21  3011573  3011574  3011575  3011576  3011577  3011578  3011579  
2017-08-22  3013355  3013356  3013357  3013358  3013359  3013360  3013361  
2017-08-23  3015137  3015138  3015139  3015140  3015141  3015142  3015143  
2017-08-24  3016919  3016920  3016921  3016922  3016923  3016924  3016925  
2017-08-25  3018701  301870

In [21]:
#submission = pd.read_csv(r'C:\Users\sahithi aharam\Downloads\sample_submission.csv', index_col='id')
#submission.shape

In [22]:
## mapping ypredict to pivoted test data
#for day_ith, day_ith_pred in y_predict.iterrows():
    #for n_samples_per_day in range(len(day_ith_pred)): ## iterating the number of sample, from 0 to 1781, for 16 days
        #sample_id = pivoted_test.iloc[[day_ith], [n_samples_per_day]].values[0][0] #total number of samples
        #values= max(0,day_ith_pred.values[n_samples_per_day]) #price that is negative will be set to 0
        #submission.at[sample_id, 'sales'] = values

In [23]:
#submission

In [24]:
#model.save('model.h5')

In [25]:
#from keras.models import load_model

In [26]:
#loaded_model = load_model('model.h5')

In [27]:
#loaded_model.summary()

In [28]:
y_predict.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Columns: 1782 entries, (1, 0) to (9, 32)
dtypes: float32(1782)
memory usage: 28.0 KB


In [29]:
model.save('model.h5')


In [30]:
from keras.models import load_model

In [31]:
loaded_model = load_model('model.h5')

In [32]:
loaded_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn (SimpleRNN)      (None, 4, 200)            396600    
                                                                 
 batch_normalization (BatchN  (None, 4, 200)           800       
 ormalization)                                                   
                                                                 
 dropout (Dropout)           (None, 4, 200)            0         
                                                                 
 simple_rnn_1 (SimpleRNN)    (None, 4, 200)            80200     
                                                                 
 batch_normalization_1 (Batc  (None, 4, 200)           800       
 hNormalization)                                                 
                                                                 
 dropout_1 (Dropout)         (None, 4, 200)            0

In [33]:
import pandas as pd

submission = pd.read_csv(r'C:\Users\sahithi aharam\Downloads\sample_submission.csv')

def predict_sales_for_date_and_product(loaded_model, date, product_id):
    num_samples_per_day = 1781  # Replace with the actual number of samples per day
    return [i * product_id for i in range(num_samples_per_day)]

# Placeholder function for demonstration, replace with your actual logic
def calculate_sample_id(date, product_id, n_samples_per_day):
    return f"{date.strftime('%Y-%m-%d')}_{product_id}_{n_samples_per_day}"

# Assuming user provides input for date, product ID, and n_samples_per_day with error handling
while True:
    input_date = input("Enter the date (YYYY-MM-DD): ")
    try:
        date_to_predict = pd.to_datetime(input_date)
        break  # Exit the loop if the date is successfully parsed
    except ValueError:
        print("Invalid date format. Please enter the date in the format YYYY-MM-DD.")

product_id = int(input("Enter the product ID: "))
n_samples_per_day = int(input("Enter the number of samples per day: "))

# Predict the value for the specified date and product ID
predicted_value = predict_sales_for_date_and_product(loaded_model, date_to_predict, product_id)[n_samples_per_day]

# Update submission DataFrame with the single predicted value
sample_id = calculate_sample_id(date_to_predict, product_id, n_samples_per_day)
submission.at[sample_id, 'sales'] = predicted_value

# Display only the updated value without index and column name
result_str = submission.loc[[sample_id], 'sales'].to_string(index=False, header=False)
print(result_str)

2829.0
